### **Evaluation du modèle**
Dans ce notebook `4_model_evaluation.ipynb`, nous allons évaluer les modèles entraînés pour obtenir une vue détaillée de leurs performances sur des données de test. 

Cela inclura des analyses de métriques de classification, des courbes ROC-AUC, et des rapports sur la précision, le rappel et le F1-score pour chaque modèle. 

Ce notebook permet de comprendre comment les modèles se comportent et où des ajustements peuvent être nécessaires.

In [1]:
# Importer les fonctions nécessaires depuis le package src
from src import load_data, load_pickle, evaluate_model, plot_confusion_matrix, plot_roc_curve

# Charger les données de test
df_test = load_data('../data/processed_data.csv')  # Assurez-vous d'avoir un ensemble de test séparé ou utilisez les mêmes données

# Vectoriser les tweets de test en utilisant le vectoriseur sauvegardé
from src.feature_engineering import load_vectorizer
vectorizer = load_vectorizer('../models/tfidf_vectorizer.pkl')
X_test = vectorizer.transform(df_test['cleaned_tweet'])
y_test = df_test['label']

# Charger le modèle entraîné
model = load_pickle('../models/svm_model.pkl')  # Remplacez par le modèle souhaité

# Évaluer le modèle
metrics = evaluate_model(model, X_test, y_test)
print("Rapport de classification :")
print(metrics['classification_report'])
print("Précision du modèle :", metrics['accuracy'])

# Afficher la matrice de confusion
conf_matrix = metrics['confusion_matrix']
classes = ['Non Suspect', 'Suspect']
plot_confusion_matrix(conf_matrix, classes)

# Tracer la courbe ROC (pour classification binaire)
plot_roc_curve(model, X_test, y_test)


ModuleNotFoundError: No module named 'src'

In [ ]:
# Importations nécessaires
import pandas as pd
import joblib
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize

In [ ]:
# Charger le modèle et le vectoriseur sauvegardés
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')
best_model = joblib.load('best_svm_model.pkl')  # On peut remplacer par le modèle souhaité

In [ ]:
# Charger les données de test (assurez-vous qu'elles sont identiques à celles utilisées dans l'entraînement)
df_test = pd.read_pickle('df_data_cleaned.pkl')

# Préparation des données de test
X_test = tfidf_vectorizer.transform(df_test['cleaned_tweet'])
y_test = df_test['label']

# Prédictions sur les données de test
y_pred = best_model.predict(X_test)

In [ ]:
# Évaluation des performances avec des métriques
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

print("Matrice de confusion :")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Calcul de la précision
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle: {accuracy:.2f}")

In [ ]:
# Visualisation de la matrice de confusion
import seaborn as sns
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="d", cmap="Blues")
plt.title("Matrice de Confusion")
plt.xlabel("Prédiction")
plt.ylabel("Réel")
plt.show()

In [ ]:
# Évaluation de la courbe ROC-AUC (pour les modèles binaires)
if len(set(y_test)) == 2:  # Vérifier si c'est une classification binaire
    y_test_binarized = label_binarize(y_test, classes=[0, 1])
    y_score = best_model.decision_function(X_test)  # Pour SVM ou Logistic Regression

    fpr, tpr, thresholds = roc_curve(y_test_binarized, y_score)
    roc_auc = auc(fpr, tpr)

    # Tracer la courbe ROC
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"Courbe ROC (AUC = {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("Taux de Faux Positifs (FPR)")
    plt.ylabel("Taux de Vrais Positifs (TPR)")
    plt.title("Courbe ROC")
    plt.legend(loc="lower right")
    plt.show()
else:
    print("La courbe ROC-AUC est uniquement disponible pour la classification binaire.")


In [ ]:

# Analyse des performances de chaque classe (optionnel pour des classes multiples)
if len(set(y_test)) > 2:
    for label in set(y_test):
        print(f"\nClasse {label} - Précision : {precision_score(y_test, y_pred, pos_label=label):.2f}")
        print(f"Classe {label} - Rappel : {recall_score(y_test, y_pred, pos_label=label):.2f}")
        print(f"Classe {label} - F1-Score : {f1_score(y_test, y_pred, pos_label=label):.2f}")


### **Explication des étapes du notebook**

1. **Chargement du modèle et du vectoriseur** :
   - Le modèle pré-entraîné et le vectoriseur TF-IDF sont chargés avec `joblib`. Assurez-vous de charger le bon modèle que vous souhaitez évaluer.

2. **Préparation des données de test** :
   - Nous chargeons les données de test et appliquons la transformation TF-IDF avec le vectoriseur sauvegardé. La variable `X_test` contient les caractéristiques du texte transformées, et `y_test` les étiquettes réelles.

3. **Prédictions et Rapport de Classification** :
   - Les prédictions sur les données de test sont obtenues avec `predict`.
   - Nous utilisons `classification_report` pour obtenir des métriques détaillées (précision, rappel, F1-score) pour chaque classe.
   - La matrice de confusion est affichée avec `confusion_matrix`, permettant d'analyser où le modèle a fait des erreurs de classification.

4. **Calcul et Affichage de la Précision** :
   - La précision globale du modèle est calculée avec `accuracy_score`, une mesure de performance générale indiquant la proportion des prédictions correctes.

5. **Visualisation de la Matrice de Confusion** :
   - La matrice de confusion est visualisée sous forme de carte thermique avec `seaborn` pour une analyse visuelle rapide.

6. **Courbe ROC-AUC** (Classification Binaire uniquement) :
   - Si les données de test comportent seulement deux classes, nous calculons et affichons la courbe ROC et l'AUC.
   - La courbe ROC illustre le compromis entre le taux de faux positifs et de vrais positifs. Une AUC proche de 1 indique une bonne performance.

7. **Analyse par Classe** (Classification Multiclasse) :
   - Pour chaque classe, la précision, le rappel et le F1-score sont calculés et affichés. Cela peut être particulièrement utile pour des données déséquilibrées ou lorsque certaines classes sont plus importantes.

### **Métriques et Visualisations pour l'évaluation**

- **Rapport de Classification** : Donne des informations par classe avec précision, rappel et F1-score.
- **Matrice de Confusion** : Montre les erreurs spécifiques faites par le modèle, indiquant quelles classes sont confondues.
- **Précision** : Fournit un aperçu général de l'exactitude du modèle.
- **Courbe ROC-AUC** : Utile pour évaluer les performances sur une tâche de classification binaire, en montrant le compromis entre le rappel et la spécificité.

### **Conclusion**

Le notebook `4_model_evaluation.ipynb` fournit une analyse complète des performances des modèles, avec des visualisations et des métriques permettant de bien comprendre les capacités de généralisation. Ces étapes permettent d’identifier les améliorations possibles et d’affiner le modèle pour de meilleures performances.